<a href="https://colab.research.google.com/github/mlussati/oficina-genai/blob/master/Dia_02_Setup_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0) Setup

In [1]:
!pip -q install --upgrade transformers accelerate sentence-transformers faiss-cpu bitsandbytes

import torch, textwrap, os, re
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

CANDIDATE_MODELS = [
#    "microsoft/Phi-3.5-mini-instruct",
#    "microsoft/Phi-3-mini-4k-instruct",
#    "microsoft/Phi-3-mini-128k-instruct",
    "microsoft/phi-2",
#    "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
]

def try_load(model_id, prefer_4bit=True):
    has_gpu = torch.cuda.is_available()
    kwargs = {}
    if has_gpu and prefer_4bit:
        kwargs = dict(
            device_map="auto",
            torch_dtype=torch.float16,
            load_in_4bit=True
        )
    elif has_gpu:
        kwargs = dict(
            device_map="auto",
            torch_dtype=torch.float16
        )
    else:
        kwargs = dict(
            device_map="auto"
        )
    tok = AutoTokenizer.from_pretrained(model_id)
    mdl = AutoModelForCausalLM.from_pretrained(model_id, **kwargs)
    gen_max = 512 if "Phi-3" in model_id or "phi-2" in model_id else 384
    pipe = pipeline("text-generation", model=mdl, tokenizer=tok, max_new_tokens=gen_max)
    return pipe, tok, model_id, gen_max

pipe = None
tok = None
model_name = None
max_new = 384

errors = []
for mid in CANDIDATE_MODELS:
    try:
        pipe, tok, model_name, max_new = try_load(mid)
        break
    except Exception as e:
        errors.append(f"{mid}: {e}")

if pipe is None:
    raise RuntimeError("Falha ao carregar todos os modelos. Detalhes:\n" + "\n".join(errors))

print("✅ Modelo carregado:", model_name)

# Formatação ChatML (Phi-3)
def format_chat_phi(system, user):
    return f"<|system|>\n{system}\n<|end|>\n<|user|>\n{user}\n<|end|>\n<|assistant|>\n"

def chat(prompt, system="Você é um assistente útil e conciso.", temperature=0.7):
    text = format_chat_phi(system, prompt)
    out = pipe(text, do_sample=True, temperature=temperature)[0]["generated_text"]
    if "<|assistant|>" in out:
        out = out.split("<|assistant|>")[-1]
    out = re.split(r"<\|end\|>|<\|user\|>|<\|system\|>", out)[0]
    return out.strip()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ Modelo carregado: microsoft/Phi-3.5-mini-instruct


## 1) LLM — Olá, Mundo

In [2]:
print("### Exemplo 1: IA Generativa para programadores")
resp = chat("Explique IA Generativa em 4 frases e dê 1 exemplo prático para programadores (ex.: documentação de funções).", temperature=0.7)
print(textwrap.fill(resp, 100))

print("\n### Exemplo 1B: Explicar código em linguagem natural")
codigo = """
def soma_pares(nums):
    return sum(n for n in nums if n % 2 == 0)
"""
prompt = f"Explique em linguagem natural (3-4 frases) o que o código Python abaixo faz e mostre um exemplo de entrada e saída.\n\n```python\n{codigo}\n```"
print(textwrap.fill(chat(prompt, temperature=0.3), 100))

### Exemplo 1: IA Generativa para programadores
A IA Generativa refere-se à criação de novo conteúdo ou solu diffusão de modelos por meio de
algoritmos de aprendizado de máquina, particularmente aprendizado profundo. Esses modelos aprendem
padrões e podem gerar novas instâncias desses padrões, como imagens ou textos, com destreza sem
intervenção humana direta. Por exemplo, na documentação de funções, a IA Generativa pode auto-gerar
exemplos de entradas e saídas, assimilando padrões lógicos e formatos, oferecendo cenários práticos
para programadores testarem e entenderem a funcionalidade da função. Este processo automatizado
melhora a qualidade e a utilidade da documentação, tornando-a mais acessível e relevante para
desenvolvedores.  Exemplo Prático: Uma função de uma API gera cartões de crédito com números
aleatórios. Usando IA Generativa, podemos criar um modelo que produz exemplos realistas de cartões
de crédito, como "4567 8901 2345 6789 01" e "123 45 6789 0123 45678", para program

## 2) Prompt — Projetando melhores respostas (algoritmos & código)

In [3]:
print("### Exemplo 2A: Prompt simples (pode ser genérico)")
print(chat("Explique o que é complexidade de algoritmo."))

print("\n### Exemplo 2B: Persona + Formato + Restrições (melhora a clareza)")
prompt = """Você é um professor de Algoritmos.
Explique o que é complexidade de algoritmo usando o exemplo de busca binária.
Mostre a diferença entre O(n) e O(log n) em 5 linhas e termine com uma tabela de 3 linhas comparando casos.
"""
print(chat(prompt))

print("\n### Exemplo 2C: Debug guiado por prompt")
codigo_bugado = """
def fatorial(n):
    if n == 0:
        return 0
    return n * fatorial(n-1)
"""
prompt_debug = f"""Você é um revisor de código.
Encontre o bug na função Python a seguir e corrija. Explique o motivo do bug em 2 frases e forneça a versão correta.
```python
{codigo_bugado}
```"""
print(chat(prompt_debug, temperature=0.2))

### Exemplo 2A: Prompt simples (pode ser genérico)
Complexidade de algoritmo refere-se à taxa de crescimento da complexidade computacional de um algoritmo, geralmente com base no tamanho da entrada. Essa medição ajuda a determinar a eficiência e escalabilidade do algoritmo em diferentes cenários.

Existem várias métricas de complexidade comumente usadas:

1. Complexidade Temporal (Temporal)
- O tempo constante (O(1)): o tempo para executar o algoritmo não depende do tamanho da entrada dada; é constante, independentemente do tamanho.
- Tempo linear (O(n)): o tempo de execução aumenta linearmente com o tamanho da entrada. Por exemplo, percorrer um array ou uma lista que tem 'n' elementos leva O(n) de tempo.
- Tempo quadrático (O(n^2)): o tempo de execução aumenta com o quadrado do tamanho da entrada. Isso frequentemente ocorre em algoritmos com laços internos, onde cada iteração pode ser necessária para todos os elementos da entrada.
- Tempo exponencial (O(2^n)): a complexidade temporar 

## 3) Alucinação — detectando e corrigindo respostas erradas (CS)

In [4]:
print("### Exemplo 3A: Pergunta factual sem contexto (pode alucinar)")
pergunta = "Quem venceu a maratona de Boston em 2015 na categoria masculina?"
print("Pergunta:", pergunta)
print("Resposta sem contexto:\n", chat(pergunta, temperature=0.2))

print("\n### Exemplo 3B: Injetando contexto factual (anti-alucinação)")
contexto = """Fatos verificados:
- Vencedor da Maratona de Boston 2015 (masculino): Lelisa Desisa (Etiópia).
- Tempo oficial: 2:09:17.
Responda apenas com base nos fatos acima.
"""
prompt_ctx = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}\nResponda de forma direta."
print("Resposta com contexto:\n", chat(prompt_ctx, temperature=0.2))

print("\n### Exemplo 3C: Exigir 'Não sei' quando não houver evidência")
prompt_nsei = """Se você não tiver certeza com base no contexto abaixo, diga "Não sei".
Contexto:
- (vazio)
Pergunta: Quem venceu a maratona de Boston em 2015 na categoria masculina?
"""
print("Resposta com regra de 'Não sei':\n", chat(prompt_nsei, temperature=0.2))

### Exemplo 3A: Pergunta factual sem contexto (pode alucinar)
Pergunta: Quem venceu a maratona de Boston em 2015 na categoria masculina?
Resposta sem contexto:
 Em 2015, a maratona de Boston na categoria masculina foi vencida por Geoffrey Ronke. Ele completou a prova em um tempo de 2 horas, 37 minutos e 12 segundos, estabelecendo um novo recorde de categoria para Boston. É importante notar que esses dados podem ser verificados através de fontes confiáveis como a página oficial da Boston Marathon ou registros de corridores.

Para mais detalhes ou para atualizações mais recentes, recomendo consultar essas fontes diretamente.

### Exemplo 3B: Injetando contexto factual (anti-alucinação)
Resposta com contexto:
 Lelisa Desisa

### Exemplo 3C: Exigir 'Não sei' quando não houver evidência
Resposta com regra de 'Não sei':
 Não sei


In [4]:
print("### Exemplo 3A: Pergunta técnica que pode induzir alucinação")
pergunta = "Qual é a função em Python que calcula a mediana automaticamente no módulo math?"
print("Pergunta:", pergunta)
print("Resposta sem contexto:\n", chat(pergunta, temperature=0.2))

print("\n### Exemplo 3B: Injetando contexto correto (anti-alucinação)")
contexto = """Fatos verificados:
- Em Python, a função de mediana está em statistics: statistics.median().
- O módulo math NÃO possui função median().
Responda apenas com base nos fatos acima.
"""
prompt_ctx = f"Contexto:\n{contexto}\n\nPergunta: {pergunta}\nResponda de forma direta."
print("Resposta com contexto:\n", chat(prompt_ctx, temperature=0.2))

print("\n### Exemplo 3C: Regra de 'Não sei' quando não houver evidência")
prompt_nsei = """Se você não tiver certeza com base no contexto abaixo, diga \"Não sei\".
Contexto:
- (vazio)
Pergunta: Qual é a função em Python que calcula a mediana automaticamente no módulo math?
"""
print("Resposta com 'Não sei':\n", chat(prompt_nsei, temperature=0.2))

### Exemplo 3A: Pergunta técnica que pode induzir alucinação
Pergunta: Qual é a função em Python que calcula a mediana automaticamente no módulo math?
Resposta sem contexto:
 Na biblioteca padrão de Python, não há uma função `math` que calcule diretamente a mediana. No entandas, você pode calcular a mediana usando a biblioteca `statistics`, que não é parte da biblioteca `math`. Veja como você pode fazer isso:

```python
import statistics

lista = [1, 2, 3, 4, 5]

# Calcule a mediana
mediana = statistics.median(lista)

print(mediana)
```

Neste exemplo, a função `statistics.median()` calcula a mediana da lista de números. Se sua lista contiver um número par de elementos, a função retornará o número que está no meio, ou se houver empate, ela calculará a média dos dois números centrais.

Se você quiser calcular a mediana sem usar a biblioteca `statistics`, você pode fazer isso com as seguintes etapas:

```python
lista = sorted(lista)
n = len(lista)

# Para um número par de elementos
if n 

## 4) Mini‑RAG — FAISS + Sentence‑Transformers (local)

In [5]:

# 4.1 Base de conhecimento simulada (CS)
docs = [
("python_loops", """Em Python, loops podem ser feitos com for ou while.
A função range() é usada para gerar sequências numéricas (ex.: range(0, 10))."""),
("git_basics", """Comandos básicos de Git: git init, git status, git add, git commit -m "msg", git push, git pull.
Use git branch e git checkout -b para criar e trocar de branch."""),
("linux_cli", """Comandos Linux úteis: ls, cd, mkdir, rm, cp, mv, cat, grep, find.
Permissões: chmod, chown. Pacotes: apt, yum (dependendo da distro)."""),
("algorithms", """Busca binária tem complexidade O(log n). Ordenação por seleção é O(n^2).
Estruturas de dados: listas, pilhas, filas, árvores, grafos."""),
("ml_basics", """Machine Learning aprende padrões a partir de dados. Overfitting ocorre quando o modelo memoriza o treino.
Regularização ajuda a reduzir overfitting.""")
]

from sentence_transformers import SentenceTransformer
import faiss, numpy as np

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
corpus_texts = [d[1] for d in docs]
corpus_emb = embedder.encode(corpus_texts, normalize_embeddings=True)
index = faiss.IndexFlatIP(corpus_emb.shape[1])
index.add(corpus_emb)

def retrieve(query, k=2):
    q_emb = embedder.encode([query], normalize_embeddings=True)
    D, I = index.search(np.array(q_emb), k)
    ctx = "\n\n".join([corpus_texts[i] for i in I[0]])
    return ctx

# Perguntas típicas de computação
user_q = "Como faço um loop em Python e uso range()? Dê um exemplo."
ctx = retrieve(user_q, k=2)

prompt_rag = f"""Use SOMENTE o contexto a seguir para responder. Se não houver informação suficiente, diga "Não sei".
Contexto:
{ctx}

Pergunta: {user_q}
Mostre um exemplo de código curto.
"""
print("Contexto recuperado (trecho):\n", ctx[:300], "...\n")
print("Resposta RAG:\n", chat(prompt_rag, temperature=0.2))

# Atividade extra: tente com perguntas sobre Git/Linux
user_q2 = "Como criar uma nova branch e trocar para ela no Git?"
ctx2 = retrieve(user_q2, k=2)
prompt_rag2 = f"""Use APENAS o contexto a seguir.
Contexto:
{ctx2}

Pergunta: {user_q2}
Responda com comandos diretos.
"""
print("\n---\nPergunta extra (Git):", user_q2)
print("Resposta RAG:\n", chat(prompt_rag2, temperature=0.2))


Contexto recuperado (trecho):
 Em Python, loops podem ser feitos com for ou while.
A função range() é usada para gerar sequências numéricas (ex.: range(0, 10)).

Busca binária tem complexidade O(log n). Ordenação por seleção é O(n^2).
Estruturas de dados: listas, pilhas, filas, árvores, grafos. ...

Resposta RAG:
 Aqui está um exemplo simples de loop em Python usando `range()`:

```python
for i in range(0, 5):  # Gerará números de 0 a 4
    print(i)
```

Este código irá imprimir os números de 0 a 4 em linhas separadas. A função `range(0, 5)` gera uma sequência de números de 0 a 4 (excluindo 5), e o loop `for` itera sobre cada número e imprime-o.

---
Pergunta extra (Git): Como criar uma nova branch e trocar para ela no Git?
Resposta RAG:
 Para criar uma nova branch e trocar para ela no Git, use os seguintмерes comandos:

```bash
git branch novabranch
git checkout novabranch
```

Ou, para criar e mudar para uma nova branch em uma única linha:

```bash
git checkout -b novabranch
```

Ess

## 5) Agents — agente de suporte técnico (calculadora, KB e docs)

In [6]:

def tool_calc(expr: str):
    try:
        return str(eval(expr, {"__builtins__": {}}))
    except Exception as e:
        return f"Erro na expressão: {e}"

KB = {
    "horarios_biblioteca": "A biblioteca abre das 8h às 20h, de segunda a sexta.",
    "wifi_campus": "A rede Wi-Fi do campus é 'UNIV-ALUNOS'; login com RA e senha do portal.",
    "docs_python_list": "https://docs.python.org/3/tutorial/datastructures.html#more-on-lists",
}

def tool_kb(key: str):
    return KB.get(key.lower(), "Não encontrei essa informação na base local.")

def tool_doc(topic: str):
    base = "https://docs.python.org/3/search.html?q="
    return base + topic.replace(" ", "+")

TOOLS = {
    "calculator": tool_calc,
    "kb_lookup": tool_kb,
    "doc_search": tool_doc,
}

AGENT_SYS = """Você é um agente que pode chamar ferramentas antes de responder.
Se precisar de cálculo use: CALL_TOOL:calculator|<expressao>
Se precisar de um fato da base local use: CALL_TOOL:kb_lookup|<chave>
Se precisar sugerir documentação oficial use: CALL_TOOL:doc_search|<termo>
Se não precisar de ferramenta, responda normalmente.
Depois de receber o RESULT_TOOL, produza a resposta final curta e correta.
Respeite o formato à risca.
"""

def agent(question):
    first = chat(
        f"Pergunta do usuário: {question}\nDecida se precisa de ferramenta. Se sim, use o formato CALL_TOOL:...",
        system=AGENT_SYS,
        temperature=0.2
    )
    if first.strip().startswith("CALL_TOOL:"):
        try:
            _, rest = first.split(":", 1)
            tool, arg = rest.split("|", 1)
            tool = tool.strip(); arg = arg.strip()
            if tool in TOOLS:
                result = TOOLS[tool](arg)
            else:
                result = f"Ferramenta desconhecida: {tool}"
        except Exception as e:
            result = f"Erro ao interpretar chamada de ferramenta: {e}"

        final = chat(
            f"Pergunta: {question}\nRESULT_TOOL ({tool}): {result}\nAgora responda de forma final e direta.",
            system=AGENT_SYS,
            temperature=0.2
        )
        return f"[{first}]\n\n→ Resultado da ferramenta: {result}\n\nResposta final: {final}"
    else:
        return f"(Sem ferramenta)\nResposta: {first}"

print("### Exemplo 5A: Cálculo (tempo em segundos)")
print(agent("Quantos segundos há em 2 horas?"))

print("\n### Exemplo 5B: Consulta à base local (Wi-Fi do campus)")
print(agent("Qual é a rede Wi-Fi do campus?"))

print("\n### Exemplo 5C: Link para documentação oficial")
print(agent("Onde aprendo sobre listas em Python?"))


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


### Exemplo 5A: Cálculo (tempo em segundos)
[CALL_TOOL:calculator|60*60*2

Resposta: 7200 segundos]

→ Resultado da ferramenta: Erro na expressão: invalid syntax (<string>, line 3)

Resposta final: Há 7200 segundos em 2 horas. (Calculado: 2 horas * 60 minutos/hora * 60 segundos/minuto = 7200 segundos)

### Exemplo 5B: Consulta à base local (Wi-Fi do campus)
[CALL_TOOL:kb_lookup|campus Wi-Fi

---

Como assistente de IA, não execute chamadas de sistema no mundo real, mas aqui está como você poderia lidar com a situação:

1. O sistema de busca da base de conhecimento (kb_lookup) procuraria informações sobre a rede Wi-Fi no campus.
2. Ele retornaria o nome da rede Wi-Fi, possivelmente juntamente com outras informações relevantes, como senha ou detalhes de conexão.
3. A informação recuperada seria então fornecida ao usuário.

Por exemplo, a resposta poderia ser: "A rede Wi-Fi do campus é 'UniNet', e sua senha é 'ExemploSenha123'. Certifique-se de usar a mesma senha que você usou para config